<a href="https://colab.research.google.com/github/muriloWeber/fuzzyLogicCase/blob/main/estudo_de_caso_fuzzy_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No contexto aqui apresentado, uma empresa de alimentos deseja saber a validade real de um de seus produtos em função da temperatura e umidade relativa do ambiente de armazenamento. A validade máxima desse produto é de 60 dias se armazenado à baixa temperatura em ambiente seco.
Para isso, você vai modelar a validade do produto com base na Temperatura (T) e Umidade Relativa (UR) do depósito. A T é um valor entre 0 e 40 ºC, a UR entre 0 e 100% e Validade de 0 a 60 dias, para ajudá-los na identificação da situação do produto que será estocado.

In [3]:
# Instalação do pacote skfuzzy
!pip install scikit-fuzzy

     |████████████████████████████████| 993 kB 4.9 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=1dc307583f9ce0c6a55ee86b4cd8ec67c5c813d127020f278a728d7f26d5cd68
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [4]:
# Fuzzificação

# importação dos pacotes que serão utilizados no código
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [5]:
# criando as variáveis do problema

temperatura = ctrl.Antecedent(np.arange(0, 41, 1), 'temperatura')
umidade = ctrl.Antecedent(np.arange(0, 101, 1), 'umidade')
validade = ctrl.Consequent(np.arange(0, 61, 1), 'validade')

In [9]:
# criando as funções de pertinência para a temperatura

temperatura['fria'] = fuzz.trapmf(temperatura.universe, [0, 0,10, 22])
temperatura['amena'] = fuzz.gaussmf(temperatura.universe, 22, 5)
temperatura['quente'] = fuzz.trapmf(temperatura.universe, [25, 30, 40, 40])

In [11]:
# criando as funções de pertinência para a umidade

umidade['baixa'] = fuzz.trapmf(umidade.universe, [0, 0, 20, 60])
umidade['média'] = fuzz.trapmf(umidade.universe, [0, 50, 60, 100])
umidade['alta'] = fuzz.trimf(umidade.universe, [60, 100, 100])

In [12]:
# Criando as funções de pertinência para a validade

validade['curta'] = fuzz.trapmf(validade.universe, [0, 0, 5, 30])
validade['aceitável'] = fuzz.trimf(validade.universe, [10, 30, 50])
validade['longa'] = fuzz.trapmf(validade.universe, [30, 50, 60, 60])

In [13]:
# Inferência Fuzzy e Defuzzyficação

# Base de Conhecimento/Regras

rule1 = ctrl.Rule(temperatura['fria'] & umidade['baixa'], validade['longa'])
rule2 = ctrl.Rule(temperatura['quente'] | umidade['alta'], validade['curta'])
rule3 = ctrl.Rule(umidade['média'], validade['aceitável'])

In [14]:
# Sistema Fuzzy e Simulação

validade_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
validade_simulador = ctrl.ControlSystemSimulation(validade_ctrl)